In [154]:
%pip install selenium
%pip install lxml
%pip install --upgrade --quiet  markdownify
%pip install langchain
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for langchain_community from https://files.pythonhosted.org/packages/82/c7/b6d7cb286e93f85b4882088ce530cc7d55a5683c9390d7fc1c0dbde846e2/langchain_community-0.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/c3/be/d0d44e092656fe7a06b55e6103cbce807cdbdee17884a5367c68c9860853/dataclasses_json-0.6.7-py3-none-any.whl.metadata
  Obtaining dependency information for httpx-sse<0.5.0,>=0.4.0 from https://files.pythonhosted.org/packages/e1/9b/a181f281f65d776426002f330c31849b86b31fc9d848db62e16f03ff739f/httpx_sse-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-settings<3.0.0,>=2.

In [210]:
from selenium import webdriver
from lxml import etree
from lxml import html 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
from markdownify import MarkdownConverter


In [33]:
chrome_options = Options()
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox") # linux only
chrome_options.add_argument("--headless=new") # for Chrome >= 109
# chrome_options.add_argument("--headless")
# chrome_options.headless = True # also works
driver = webdriver.Chrome(options=chrome_options)

In [107]:
URL = "https://honkai-star-rail.fandom.com/wiki/Aventurine/Lore"
DOMAIN = "https://" + URL.split("/")[2]

print(DOMAIN)

https://honkai-star-rail.fandom.com


In [142]:
def get_etree_from(URL):
    driver.get(URL)
    tree = etree.HTML(driver.page_source)
    
    return tree

def get_mission_links(tree):
    content = tree.xpath("//div[@class='mw-parser-output']")
    content = content[0]

    mission_links = []
    add_next = False

    for item in content:
        if item.tag == "h3":
            header = item.xpath(".//text()")
            header = header[0]
            if header in ['Trailblaze Missions', 'Trailblaze Continuances']:
                add_next = True
            else:
                add_next = False

        if add_next:
            mission_links.extend(item.xpath(".//li/ul//a/@href"))
        
    return mission_links

In [115]:
tree = get_etree_from(URL)

In [143]:
mission_links = get_mission_links(tree)
print(mission_links)

['/wiki/The_Devil_in_Velvet', '/wiki/Lost_Generation', '/wiki/Double_Indemnity', '/wiki/Heaven_is_a_Place_on_Earth', '/wiki/All_the_Sad_Tales', '/wiki/A_Walk_Among_the_Tombstones', '/wiki/A_Thousand_Bells_at_Dawn', '/wiki/The_Sun_Also_Rises', '/wiki/And_on_the_Eighth_Day', '/wiki/The_Knocking_at_Ungodly_Hours', '/wiki/The_Youths_Who_Chase_Dreams', '/wiki/Gentleness,_the_Name_of_Nocturne', '/wiki/Whodunit', '/wiki/Future_Market_(II)']


In [144]:
def join_domain(links):
    for i, value in enumerate(links):
        links[i] = DOMAIN + value
    return links

In [145]:
mission_links = join_domain(mission_links)
print(mission_links)

['https://honkai-star-rail.fandom.com/wiki/The_Devil_in_Velvet', 'https://honkai-star-rail.fandom.com/wiki/Lost_Generation', 'https://honkai-star-rail.fandom.com/wiki/Double_Indemnity', 'https://honkai-star-rail.fandom.com/wiki/Heaven_is_a_Place_on_Earth', 'https://honkai-star-rail.fandom.com/wiki/All_the_Sad_Tales', 'https://honkai-star-rail.fandom.com/wiki/A_Walk_Among_the_Tombstones', 'https://honkai-star-rail.fandom.com/wiki/A_Thousand_Bells_at_Dawn', 'https://honkai-star-rail.fandom.com/wiki/The_Sun_Also_Rises', 'https://honkai-star-rail.fandom.com/wiki/And_on_the_Eighth_Day', 'https://honkai-star-rail.fandom.com/wiki/The_Knocking_at_Ungodly_Hours', 'https://honkai-star-rail.fandom.com/wiki/The_Youths_Who_Chase_Dreams', 'https://honkai-star-rail.fandom.com/wiki/Gentleness,_the_Name_of_Nocturne', 'https://honkai-star-rail.fandom.com/wiki/Whodunit', 'https://honkai-star-rail.fandom.com/wiki/Future_Market_(II)']


In [212]:
def md(soup, **options):
    return MarkdownConverter(**options).convert_soup(soup)

scraped_texts = []

for link in mission_links:
    print("Scraping: "+link)
    driver.get(link)
    page_source = driver.page_source
    
    soup = BeautifulSoup(page_source)
    for e in soup.select('img'):
        e.decompose()
    for e in soup.select('.toc'):
        e.decompose()
    for e in soup.select('table'):
        e.decompose()
    to_markdown = soup.select(".mw-parser-output")
    
    text = md(to_markdown[0], strip=["a","aside"])
    scraped_texts.append([link, text])

Scraping: https://honkai-star-rail.fandom.com/wiki/The_Devil_in_Velvet
Scraping: https://honkai-star-rail.fandom.com/wiki/Lost_Generation
Scraping: https://honkai-star-rail.fandom.com/wiki/Double_Indemnity
Scraping: https://honkai-star-rail.fandom.com/wiki/Heaven_is_a_Place_on_Earth
Scraping: https://honkai-star-rail.fandom.com/wiki/All_the_Sad_Tales
Scraping: https://honkai-star-rail.fandom.com/wiki/A_Walk_Among_the_Tombstones
Scraping: https://honkai-star-rail.fandom.com/wiki/A_Thousand_Bells_at_Dawn
Scraping: https://honkai-star-rail.fandom.com/wiki/The_Sun_Also_Rises
Scraping: https://honkai-star-rail.fandom.com/wiki/And_on_the_Eighth_Day
Scraping: https://honkai-star-rail.fandom.com/wiki/The_Knocking_at_Ungodly_Hours
Scraping: https://honkai-star-rail.fandom.com/wiki/The_Youths_Who_Chase_Dreams
Scraping: https://honkai-star-rail.fandom.com/wiki/Gentleness,_the_Name_of_Nocturne
Scraping: https://honkai-star-rail.fandom.com/wiki/Whodunit
Scraping: https://honkai-star-rail.fandom.com

In [213]:
for item in scraped_texts:
    print(item)

['https://honkai-star-rail.fandom.com/wiki/The_Devil_in_Velvet', '\n\n\n\nThe Devil in Velvet\n-------------------\n\n\n\n\n\n\n\n\n### Mission Type\n\n\n Trailblaze Mission\n\n\n### Chapter\n\n\nCat Among Pigeons\n\n\n### Requirements\n\n\nTrailblaze Mission *Whodunit* completed\n\n\n### Starting Location(s)\n\n\nPenacony \\- The Reverie (Dreamscape)\n\n\n\n\n### Previous Mission\n\n\n### Next Mission\n\n\n\n\nWhodunit\nLost Generation\n\n\n\n\n\n\n* Rewards\n* Summary\n* Characters\n\n\n\n\n\n 600 80 8 300 50,000\n\n\n\n\nUsing Robin\'s death as an excuse, Aventurine invites you to collaborate with him in bringing down The Family. It is not coercion, but a temptation with the promise of gain. Although he leaves you with enough room to decide, you know that one should not be hasty when it comes to decision\\-making.\n\n\n\n\nAcheron, Aventurine, Black Swan, Dan Heng, Duke Inferno, Himeko, March 7th, Sam, Trailblazer, Welt\n\n\n\n\n\n***The Devil in Velvet*** is the first part of the T

In [215]:
import pandas as pd
df = pd.DataFrame(scraped_texts)

In [221]:
import os
path = os.getcwd()

#df.to_csv(path+"/outputs/Aventurine_lore.csv")
df.to_excel(path+"/outputs/Aventurine_lore.xlsx")